In [1]:
%load_ext autoreload
%autoreload 2

from decouple import config
from qdrant_client import QdrantClient


QDRANT_URL = config('QDRANT_URL')
QDRANT_API_KEY = config('QDRANT_API_KEY')

In [2]:
from rag.utils import load_datasets


corpus_df, queries_df, qrels_df = load_datasets(10_000, 10_000)

corpus_df.size, queries_df.size, qrels_df.size

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

(1233, 1446, 1473)

In [2]:
qdrant_client = QdrantClient(
    url=QDRANT_URL, 
    api_key=QDRANT_API_KEY,
)

print(qdrant_client.get_collections())

collections=[CollectionDescription(name='collection_1')]


In [4]:
from fastembed import (
    TextEmbedding, 
    SparseTextEmbedding, 
    LateInteractionTextEmbedding
)
from qdrant_client.models import (
    Distance, 
    VectorParams, 
    SparseVectorParams,
    SparseIndexParams,
    MultiVectorConfig,
    MultiVectorComparator,
    Modifier,
    HnswConfigDiff
)

from rag.models.config import (
    DenseModelConfig, 
    SparseModelConfig,
    RerankingModelConfig
)


dense_model_name = 'BAAI/bge-small-en-v1.5'
dense_model = TextEmbedding(model_name=dense_model_name)

sparse_model_name = 'Qdrant/bm25'
sparse_model = SparseTextEmbedding(
    model_name=sparse_model_name,
    k1=1.5,
    b=0.75
)

reranking_model_name = 'colbert-ir/colbertv2.0'
reranking_model = LateInteractionTextEmbedding(reranking_model_name)

dense_model_config = DenseModelConfig(
    name=dense_model_name,
    vector_params=VectorParams(
        size=384,
        distance=Distance.COSINE,
        hnsw_config=HnswConfigDiff(
            m=16,
            ef_construct=128,
            on_disk=True
        ),
        on_disk=True
    )
)

sparse_model_config = SparseModelConfig(
    name=sparse_model_name,
    sparse_vector_params=SparseVectorParams(
        index=SparseIndexParams(
            on_disk=True
        ),
        modifier=Modifier.IDF
    )
)

reranking_model_config = RerankingModelConfig(
    name=reranking_model_name,
    vector_params=VectorParams(
        size=100,
        distance=Distance.COSINE,
        hnsw_config=HnswConfigDiff(
            m=0     # disable HNSW
        ),
        on_disk=True,
        multivector_config=MultiVectorConfig(
            comparator=MultiVectorComparator.MAX_SIM,
        )
    )
)

In [5]:
from rag.models import (
    DenseSearch,
    SparseSearch,
    HybridSearch, 
    HybridRerankingSearch, 
    Metadata
)

In [6]:
documents = [
    'FastEmbed is lighter than Transformers & Sentence-Transformers.',
    'FastEmbed is supported by and maintained by Qdrant.',
]
metadatas = [
    Metadata(
        id=i + 1,
        text=document
    )
    for i, document in enumerate(documents)
]

dense_embeddings = list(dense_model.embed(documents))
sparse_embeddings = list(sparse_model.embed(documents))
reranking_embeddings = list(reranking_model.embed(documents))

In [ ]:
dense_search = DenseSearch(
    qdrant_client=qdrant_client, 
    dense_model_config=dense_model_config
)
dense_collection_name = 'dense_collection'

dense_search.create_collection(dense_collection_name)
dense_search.upsert(
    dense_collection_name,
    dense_embeddings,
    metadatas
)

In [ ]:
sparse_search = SparseSearch(
    qdrant_client=qdrant_client, 
    sparse_model_config=sparse_model_config
)
sparse_collection_name = 'sparse_collection'

sparse_search.create_collection(sparse_collection_name)
sparse_search.upsert(
    sparse_collection_name,
    sparse_embeddings,
    metadatas
)

In [ ]:
hybrid_search = HybridSearch(
    qdrant_client=qdrant_client, 
    dense_model_config=dense_model_config,
    sparse_model_config=sparse_model_config
)
hybrid_collection_name = 'hybrid_collection'

hybrid_search.create_collection(hybrid_collection_name)
hybrid_search.upsert(
    hybrid_collection_name, 
    dense_embeddings, 
    sparse_embeddings,
    metadatas
)

In [8]:
query_document = 'Who is lighter than Transformers?'

query_dense_embedding = next(iter(dense_model.embed(query_document)))
query_sparse_embedding = next(iter(sparse_model.embed(query_document)))
query_reranking_embedding = next(iter(reranking_model.embed(query_document)))

In [ ]:
hybrid_search.search(
    hybrid_collection_name,
    query_dense_embedding,
    query_sparse_embedding,
    1
)

[ScoredPoint(id=1, version=0, score=1.0, payload={'text': 'FastEmbed is lighter than Transformers & Sentence-Transformers.'}, vector=None, shard_key=None, order_value=None)]

In [ ]:
import json
from fastembed.sparse import SparseEmbedding
from tokenizers import Tokenizer


def get_tokens_and_weights(sparse_embedding: SparseEmbedding, tokenizer: Tokenizer) -> dict:
    token_weight_dict = {}
    
    for i in range(len(sparse_embedding.indices)):
        token = tokenizer.decode([sparse_embedding.indices[i]])
        weight = sparse_embedding.values[i]
        token_weight_dict[token] = weight

    # sort by weights
    token_weight_dict = dict(sorted(token_weight_dict.items(), key=lambda item: item[1], reverse=True))
    
    return token_weight_dict


tokenizer = Tokenizer.from_pretrained('Qdrant/SPLADE_PP_en_v1')
index = 0
print(json.dumps(get_tokens_and_weights(sparse_embeddings[index], tokenizer), indent=4))

In [ ]:
# TODO add nvidia API for LLM